In [4]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score,precision_score, recall_score, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
import tensorflow.keras
# import keras.metrics
# from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D
# from keras.models import Sequential
# from keras.callbacks import History 
# from keras.utils import plot_model
# from keras.optimizers import SGD
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV, RFE


from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import pandas as pd
import os
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import LeaveOneOut
from imblearn.over_sampling import SMOTE, ADASYN
#from imblearn.pipeline import Pipeline
from xgboost import XGBClassifier

In [5]:
path = "PTSD.xlsx"
df = pd.read_excel(path)
df = df[~df["PCL_Strict3"].isna()]


In [23]:
features = ['instrumental_support1', 'highschool_diploma', 'positive_reframing2', 'T1bias', 'PCL2', 'planning2', 'HL_MAOA', 'active_coping1', 'acceptance2', 'emotional_support1', 'COMT_Ranked', 'COMT_Hap1_LvsMH']


In [24]:

numerical_features = [ "T1Acc1t", "T1Acc1n", "T1bias", "state1", "trait1", "lot1", "phq1",
                      "cd_risc1", "PCL1", "PCL2", "trauma_history8_1"]
categorical_features = ["age", "highschool_diploma",  "dyslexia", "ADHD", "T1Acc1t", "T1Acc1n", "T1bias", "phq1", 
                        "lot1", "trait1", "state1",  "PCL_Broad1", "PCL_Strict1", "active_coping1",
                        "planning1", "positive_reframing1", "acceptance1", "humor1", "religion1", 
                        "emotional_support1","instrumental_support1", "self_distraction1", "denial1", 
                        "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1"]

#df['bad_features'] = (df > df.mean())[bad_features].sum(axis=1)

# imp = SimpleImputer(missing_values=np.nan, strategy='mean')
# df[numerical_features] = imp.fit_transform(df[numerical_features])

# imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
# df[categorical_features] = imp.fit_transform(df[categorical_features])


X = df[features]


# ss = StandardScaler()
# X = ss.fit_transform(X)


Y = df["PCL_Strict3"]

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1, random_state=271828, stratify=Y)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_train, y_train, test_size = 0.1, random_state=271828, stratify=y_train)


In [26]:
pipe =  Pipeline([
                ('classifier', XGBClassifier())])

In [27]:
params_grid = [
    {
       # 'pca__n_components': [0.9],
        'classifier__min_child_weight': [1, 2],
        'classifier__gamma': [0, 0.5],
        'classifier__subsample': [0.9],
        'classifier__colsample_bytree': [0.9, 0.7],
        'classifier__max_depth': [3, 7],
        'classifier__reg_alpha' : [0.5],
        'classifier__reg_lambda' : [0.8],
        'classifier__scale_pos_weight' : [0.9, 1, 1.1],
    'classifier__learning_rate': [0.15, 0.2, 0.1, 0.05], #so called `eta` value
              'classifier__n_estimators': [150, 250, 100, 350] #number of trees, change it to 1000 for better results
    }]

In [28]:
cv = StratifiedKFold(3)
gs = GridSearchCV(pipe, params_grid, cv=cv, scoring='precision')
gs.fit(X_train_2, y_train_2);

gs.best_params_

C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'classifier__colsample_bytree': 0.9,
 'classifier__gamma': 0,
 'classifier__learning_rate': 0.1,
 'classifier__max_depth': 3,
 'classifier__min_child_weight': 1,
 'classifier__n_estimators': 250,
 'classifier__reg_alpha': 0.5,
 'classifier__reg_lambda': 0.8,
 'classifier__scale_pos_weight': 1,
 'classifier__subsample': 0.9}

In [29]:
gs.best_score_

0.3993174061433447

In [30]:
results = []
score_list = []
k = 50
for i in range (k):
    kfold = StratifiedKFold(n_splits=3, shuffle=True)
    
    score = cross_val_score(gs.best_estimator_, X_train_2, y_train_2, cv=kfold, scoring='precision')
    results.append(score.mean())
    current_scores = [i for i in score]
    #score_list.extend(current_scores)
    #print(current_scores)
    #print(score.mean())
sum(results)/k

C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 

0.23587037037037034

In [31]:
repr(score_list)

'[]'

In [15]:
x = [0.0, 0.25, 0.25, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 1.0, 0.0]




sum(x)/len(x)

0.17948717948717946